In [2]:
%matplotlib widget

import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_plate_number,
    get_postion_number,
)

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import (
    run_parallel,
    find_state,
    directory_scratch,
    directory_project,
    path_code,
)
from amftrack.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter
from scipy.ndimage import uniform_filter1d
from scipy import stats
import statsmodels.api as sm
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

window = 800
infos = pickle.load(
    open(f"{path_code}/MscThesis/Results/straight_bait_{window}.pick", "rb")
)


def get_angle(xa, ya, xb, yb):
    dot_product = (xa * xb + ya * yb) / np.sqrt(
        (xa**2 + ya**2) * (xb**2 + yb**2)
    )
    angle = (np.arccos(dot_product) / (2 * np.pi) * 360) * (
        1 - 2 * ((ya * xb - yb * xa) >= 0)
    )
    return angle


infos["abs_angle_to_P"] = np.abs(infos["angle_to_P"])
infos["abs_curvature"] = np.abs(infos["curvature"])
infos["curvature_scaled"] = infos["curvature"] * np.sqrt(infos["growth"])
infos["abs_curvature_sq"] = np.abs(infos["curvature"]) * np.sqrt(infos["growth"])
infos["signed_straight"] = (1 - infos["straightness"]) * (1 - 2 * (infos["angle"] < 0))
infos["inv_dens"] = 1 / infos["density"]
infos["side_cross"] = infos.apply(
    lambda row: comments[row["plate"]] if row["plate"] in comments.keys() else "None",
    axis=1,
)
infos["x"] = infos["x"].astype(float)
infos["vx"] = infos["vx"].astype(float)
infos["y"] = infos["y"].astype(float)
infos["vy"] = infos["vy"].astype(float)
infos["xinit"] = infos["xinit"].astype(float)
infos["yinit"] = infos["yinit"].astype(float)
blur = 20
infos["v"] = np.sqrt((infos["vx"] ** 2 + infos["vy"] ** 2).astype(float))
infos["gd"] = np.sqrt(
    (infos[f"grad_density_x{blur}"] ** 2 + infos[f"grad_density_y{blur}"] ** 2).astype(
        float
    )
)
infos["spvgd"] = (
    infos["vx"] * infos[f"grad_density_x{blur}"]
    + infos["vy"] * infos[f"grad_density_y{blur}"]
) / (infos["v"] * infos["gd"])
infos["angle_vgd"] = get_angle(
    -infos[f"grad_density_x{blur}"],
    -infos[f"grad_density_y{blur}"],
    infos["vx"],
    infos["vy"],
)
infos["angle_vgd2"] = get_angle(
    -infos[f"grad_density_x{blur}"],
    -infos[f"grad_density_y{blur}"],
    infos["xinit"],
    infos["yinit"],
)
infos["angle_Pgd"] = infos["angle_vgd"] + infos["angle_to_P"]
infos["angle_Ngd"] = infos["angle_vgd"] + infos["angle_to_N"]
# infos['residual']=infos['speed']-f(infos['spvgd'])
corrected = infos.loc[
    (infos["straightness"] <= 1)
    & (infos["speed"] >= 25)
    & (infos["speed"] < 400)
    & (infos["straightness"] > 0.95)
    & (infos["density"] > 0)
]
baits = corrected.loc[corrected["treatment"] == "baits"]
no_baits = corrected.loc[corrected["treatment"] == "25"]
no_baits_new = corrected.loc[corrected["treatment"] == "25*"]

In [3]:
def getKernelDensityEstimation(values, x, bandwidth=0.2, kernel="gaussian"):
    model = KernelDensity(kernel=kernel, bandwidth=bandwidth)
    model.fit(values[:, np.newaxis])
    log_density = model.score_samples(x[:, np.newaxis])
    return np.exp(log_density)


def bestBandwidth(data, minBandwidth=0.1, maxBandwidth=2, nb_bandwidths=30, cv=30):
    """
    Run a cross validation grid search to identify the optimal bandwidth for the kernel density
    estimation.
    """
    from sklearn.model_selection import GridSearchCV

    model = GridSearchCV(
        KernelDensity(),
        {"bandwidth": np.linspace(minBandwidth, maxBandwidth, nb_bandwidths)},
        cv=cv,
    )
    model.fit(data[:, None])
    return model.best_params_["bandwidth"]

In [5]:
corrected

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,dist_P,...,signed_straight,inv_dens,side_cross,v,gd,spvgd,angle_vgd,angle_vgd2,angle_Pgd,angle_Ngd
0,152,"(32, 160, 190)",25*,-3.562280,-0.001558,1.074638,2286.107090,285.763386,0.992977,20492.781760,...,-0.007023,0.930546,None,1315.971504,0.000201,-0.986268,-9.506115,-7.096098,-60.386044,54.164263
1,152,"(32, 160, 190)",25*,-9.275865,-0.007889,0.754348,1175.807811,293.951953,0.996975,19141.690142,...,-0.003025,1.325648,None,679.566038,0.000299,-0.957944,-16.675842,-13.526437,-67.572989,55.093304
2,152,"(32, 160, 190)",25*,-36.035662,-0.047718,0.698551,755.181576,188.795394,0.960255,18424.804782,...,-0.039745,1.431535,None,420.386727,0.000320,-0.983630,-10.381496,-13.400523,-69.128035,57.933718
3,152,"(32, 160, 190)",25*,-0.889220,-0.000803,1.898551,1106.739843,276.684961,0.990458,20015.513961,...,-0.009542,0.526718,None,635.465971,0.000183,-0.858050,-30.901647,-29.683530,-17.330733,91.128020
4,152,"(32, 160, 190)",25*,11.269844,0.010129,1.440580,1112.628557,278.157139,0.980560,19636.413828,...,0.019440,0.694165,None,632.463438,0.000194,-0.984485,10.105970,7.229420,1.530880,107.973989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15221,424,"(13, 116, 131)",baits,7.281669,0.012852,0.901449,566.586163,141.646541,0.968746,47451.023860,...,0.031254,1.109325,left,318.190195,0.000383,0.438729,-116.022832,-117.221523,-76.281616,-178.705634
15222,424,"(13, 116, 131)",baits,-20.905807,-0.030679,1.987681,681.432714,170.358178,0.984990,46972.393814,...,-0.015010,0.503099,left,389.104099,0.000430,0.309853,-108.050391,-100.927957,-72.735319,-177.386033
15224,424,"(13, 116, 131)",baits,24.176796,0.050994,1.956522,474.114541,118.528635,0.992319,45906.669249,...,0.007681,0.511111,left,272.737969,0.000449,-0.186696,79.239947,62.903977,-55.542194,191.549779
15225,424,"(13, 116, 131)",baits,16.548582,0.029624,2.842029,558.616681,139.654170,0.981037,46240.456212,...,0.018963,0.351861,left,317.694822,0.000443,0.063736,93.654256,82.548061,-59.452439,190.426191


In [6]:
plt.close("all")
plt.rcParams.update(
    {"font.family": "verdana", "font.weight": "normal", "font.size": 20}
)
plate_select = corrected.loc[corrected["treatment"] == "25*"]
# plate_select = corrected.loc[corrected ['plate'].isin(right_crossing)]
# straight_going = corrected.loc[corrected['plate'].isin(non_turning)]
abcisse = "density"
ordinate = "speed"
tab = plate_select
baits_sort = tab.sort_values(abcisse)
N = 400
moving_av = baits_sort.rolling(N, min_periods=N // 2).mean()
moving_std = baits_sort.rolling(N, min_periods=N // 2).std()

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.set_xlabel("density($mm.mm^{-2}$)")

ax.set_ylabel("hyphal tip speed ($\mu m .h^{-1}$)")
ax.set_xlim(0, 6)
# ax.set_ylim(-0.5,0.5)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
for plate in set(tab["plate"].values):
    select = tab.loc[tab["plate"] == plate]
    ax.scatter(select[abcisse], select[ordinate], label=plate, alpha=0.3)
ax.plot(moving_av[abcisse], moving_av[ordinate], color="green", label="moving average")
ax.plot(
    moving_av[abcisse],
    (moving_av[ordinate] + moving_std[ordinate] / np.sqrt(N)),
    color="red",
    label="std",
)
ax.plot(
    moving_av[abcisse],
    (moving_av[ordinate] - moving_std[ordinate] / np.sqrt(N)),
    color="red",
    label="std",
)
# ax.legend()

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:1377: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:237: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.newaxis]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:239: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


In [4]:
plt.close("all")
plt.rcParams.update(
    {"font.family": "verdana", "font.weight": "normal", "font.size": 20}
)
plate_select = corrected.loc[corrected["plate"] == 94]
# plate_select = corrected.loc[corrected ['plate'].isin(right_crossing)]
# straight_going = corrected.loc[corrected['plate'].isin(non_turning)]
abcisse = "spvgd"
ordinate = "speed"
tab = plate_select
baits_sort = tab.sort_values(abcisse)
N = 400
moving_av = baits_sort.rolling(N, min_periods=N // 2).mean()
moving_std = baits_sort.rolling(N, min_periods=N // 2).std()

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.set_ylabel("speed ($\mu m .h^{-1}$)")

ax.set_xlabel("normalized scalar product speed.density gradient")
ax.set_xlim(-1, 1)
# ax.set_ylim(-0.5,0.5)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
for plate in set(tab["plate"].values):
    select = tab.loc[tab["plate"] == plate]
    ax.scatter(select[abcisse], select[ordinate], label=plate, alpha=0.3)
ax.plot(moving_av[abcisse], moving_av[ordinate], color="green", label="moving average")
ax.plot(
    moving_av[abcisse],
    (moving_av[ordinate] + moving_std[ordinate] / np.sqrt(N)),
    color="red",
    label="std",
)
ax.plot(
    moving_av[abcisse],
    (moving_av[ordinate] - moving_std[ordinate] / np.sqrt(N)),
    color="red",
    label="std",
)
# ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
from statsmodels.nonparametric.bandwidths import (
    bw_silverman,
    bw_scott,
    select_bandwidth,
)

dummy_baits = [298, 319, 373, 344]
P_N_crossing_P = [420, 433, 405, 425, 384]
P_N_crossing_N = [423, 436, 409, 400, 403]
plate_select = corrected.loc[corrected["plate"].isin([94])]
# plate_select = corrected.loc[corrected ['treatment']=='25*']
plt.rcParams.update(
    {"font.family": "verdana", "font.weight": "normal", "font.size": 20}
)
plt.close("all")
fig = plt.figure()
bins = np.linspace(0, 400, 50)
ax = fig.add_subplot(111)
# ax.hist(corrected.loc[corrected['plate'].isin(dummy_baits)]['speed'],bins,alpha=0.25,label='Dummy baits (homogeneous soluble)',density=True)
# ax.hist(corrected.loc[corrected['plate'].isin(P_N_crossing_P)]['speed'],bins,alpha=0.25,label='P&N baits crossing P (heterogeneous rock form)',density=True)
# ax.hist(corrected.loc[corrected['plate'].isin(P_N_crossing_N)]['speed'],bins,alpha=0.25,label='P&N baits crossing N (heterogeneous rock form)',density=True)
# ax.hist(corrected.loc[corrected ['treatment']=='25*']['speed'],bins,alpha=0.25,label='No baits (homogeneous soluble)',density=True)
# # ax.hist(corrected.loc[corrected['plate'] == 69]['speed'],bins,alpha=0.3,label='No baits (homogeneous soluble)',density=True)
# # ax.hist(corrected.loc[corrected['plate'] == 102]['speed'],bins,alpha=0.3,label='No baits (homogeneous soluble)',density=True)
# # x = np.linspace(-6,6,100)
x = np.linspace(0, 400, 100)

bandwidth = 5
data = corrected.loc[corrected["plate"] == 94]["speed"]
# data = corrected.loc[corrected['treatment'] == '25*']['speed']

# data = plate_select['speed']

cv_bandwidth = bestBandwidth(data, minBandwidth=2, maxBandwidth=20)
# cv_bandwidth = 4


kde = getKernelDensityEstimation(data, x, bandwidth=cv_bandwidth)
ax.plot(x, kde, alpha=0.8, label=f"bandwidth = {round(cv_bandwidth, 2)}")
# ax.plot(x, stats_models_cv, alpha = 0.8, label = f'bandwidth = {round(cv_bandwidth, 2)}')
ax.hist(data, 30, density=True)
ax.set_xlabel(r"speed ($\mu m.h^{-1} $)")
# plt.legend(loc='upper right')
ax.set_ylabel(r"distribution density")
plt.savefig("plot.eps", format="eps")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
from statsmodels.nonparametric.bandwidths import (
    bw_silverman,
    bw_scott,
    select_bandwidth,
)

dummy_baits = [298, 319, 373, 344]
P_N_crossing_P = [420, 433, 405, 425, 384]
P_N_crossing_N = [423, 436, 409, 400, 403]
plate_select = corrected.loc[corrected["plate"].isin([344])]
# plate_select = corrected.loc[corrected ['treatment']=='25*']

plt.close("all")
fig = plt.figure()
bins = np.linspace(0, 400, 50)
ax = fig.add_subplot(111)
# ax.hist(corrected.loc[corrected['plate'].isin(dummy_baits)]['speed'],bins,alpha=0.25,label='Dummy baits (homogeneous soluble)',density=True)
# ax.hist(corrected.loc[corrected['plate'].isin(P_N_crossing_P)]['speed'],bins,alpha=0.25,label='P&N baits crossing P (heterogeneous rock form)',density=True)
# ax.hist(corrected.loc[corrected['plate'].isin(P_N_crossing_N)]['speed'],bins,alpha=0.25,label='P&N baits crossing N (heterogeneous rock form)',density=True)
# ax.hist(corrected.loc[corrected ['treatment']=='25*']['speed'],bins,alpha=0.25,label='No baits (homogeneous soluble)',density=True)
# # ax.hist(corrected.loc[corrected['plate'] == 69]['speed'],bins,alpha=0.3,label='No baits (homogeneous soluble)',density=True)
# # ax.hist(corrected.loc[corrected['plate'] == 102]['speed'],bins,alpha=0.3,label='No baits (homogeneous soluble)',density=True)
# # x = np.linspace(-6,6,100)
x = np.linspace(0, 400, 100)

bandwidth = 5
data = corrected.loc[corrected["plate"] == 94]["speed"]
# data = plate_select['speed']

cv_bandwidth = bestBandwidth(data, minBandwidth=2, maxBandwidth=20)
# cv_bandwidth = 4


kde = getKernelDensityEstimation(data, x, bandwidth=cv_bandwidth)
ax.plot(x, kde, alpha=0.8, label=f"bandwidth = {round(cv_bandwidth, 2)}")
# ax.plot(x, stats_models_cv, alpha = 0.8, label = f'bandwidth = {round(cv_bandwidth, 2)}')
ax.hist(data, 30, density=True)
ax.set_xlabel(r"speed ($\mu m.h^{-1} $)")
plt.legend(loc="upper right")
ax.set_ylabel(r"density")
plt.savefig("plot.eps", format="eps")

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
data

7362    251.281755
7363    210.899296
7364    215.332326
7366    224.802515
7367    215.776253
           ...    
7924    234.845424
7925    291.472630
7926    154.092058
7927    233.436906
7928    194.557777
Name: speed, Length: 428, dtype: float64

In [13]:
from statsmodels.nonparametric.kernel_density import KDEMultivariate

stats_models_cv = KDEMultivariate(data, "c", bw="cv_ml").pdf(x)

In [19]:
plt.close("all")
plt.rcParams.update(
    {"font.family": "verdana", "font.weight": "normal", "font.size": 20}
)
plate_select = corrected.loc[corrected["treatment"] == "25*"]
# plate_select = corrected.loc[corrected ['plate'].isin(right_crossing)]
# straight_going = corrected.loc[corrected['plate'].isin(non_turning)]
abcisse = "density"
ordinate = "speed"
tab = plate_select
baits_sort = tab.sort_values(abcisse)
N = 400
moving_av = baits_sort.rolling(N, min_periods=N // 2).mean()
moving_std = baits_sort.rolling(N, min_periods=N // 2).std()

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.set_xlabel("fungal density($mm.mm^{-2}$)")

ax.set_ylabel("hyphal tip speed ($\mu m .h^{-1}$)")
ax.set_xlim(0, 6)
# ax.set_ylim(-0.5,0.5)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
for plate in set(tab["plate"].values):
    select = tab.loc[tab["plate"] == plate]
    plate_select_pla = corrected.loc[corrected["plate"] == plate]
    # plate_select = corrected.loc[corrected ['plate'].isin(right_crossing)]
    # straight_going = corrected.loc[corrected['plate'].isin(non_turning)]
    abcisse = "density"
    ordinate = "speed"
    tab = plate_select_pla
    baits_sort = tab.sort_values(abcisse)
    N = 100
    moving_av_pla = baits_sort.rolling(N, min_periods=N // 2).mean()
    moving_std_pla = baits_sort.rolling(N, min_periods=N // 2).std()
    ax.scatter(
        plate_select_pla[abcisse], plate_select_pla[ordinate], label=plate, alpha=0.1
    )
    ax.plot(
        moving_av_pla[abcisse],
        moving_av_pla[ordinate],
        label="moving average",
        linestyle=(0, (3, 1, 1, 1)),
    )

ax.plot(moving_av[abcisse], moving_av[ordinate], color="green", label="moving average")
ax.plot(
    moving_av[abcisse],
    (moving_av[ordinate] + moving_std[ordinate] / np.sqrt(N)),
    color="red",
    label="std",
)
ax.plot(
    moving_av[abcisse],
    (moving_av[ordinate] - moving_std[ordinate] / np.sqrt(N)),
    color="red",
    label="std",
)

# ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …